# Demonstration of interactive equation plotting

This notebook demostrates using Bokeh and callbacks to generate an interactive plot of $x^{i}$, with the parameter i being adjustable by the user via slider. This is achieved by initially plotting a bokeh plot with data source of x being equally spaced points from 0 to 10 and y being sin(x). When the user adjusts the slider, a CustomJS callback is execute, updating the value of y to being sin(x)^i, automatically updating the plot.

This is significant as it demonstrates the use of callbacks and live updating of data sources without running an external Bokeh server or a standalone HTML file

# Libraries Required

We will need numpy for MATLAB-like nuemrical operations and bokeh for plotting.

In [1]:
#this script tests the JS callback functions using Bokeh to produce an interactive plot of y = (x-1)^i + x^(i-1)
#for different values of i
import numpy as np
from bokeh.models import CustomJS, Slider, ColumnDataSource
import bokeh.models.widgets
from bokeh.models.widgets import Select
from bokeh.plotting import output_file, show, output_notebook, figure
from bokeh.io import vform
output_notebook()

Loading BokehJS ...

# Initialization of data source

First, we define x and y vector-like variable to initally have values for our desired function to be adjusted later.

In [2]:
# defining data source and x, y variables to plot

# initial value for x and y vectors
start = 0
end = 10
num_points = 1000
x = np.linspace(start, end, num_points)
y = np.power(x,1)
# defining data source for Bokeh's ColumnDataSource data type
source = ColumnDataSource(data=dict(x=x, y=y))

# Using Bokeh to plot

We create a bokeh figure using the x and y vectors created with numpy and adjust the parameters of our plot to look nicer.

In [3]:
# plotting and adjusting figure properties
p = figure(plot_width=500, plot_height=500, title="Plot of x^i", toolbar_location="below", tools="pan,wheel_zoom,box_zoom,reset,resize")
p.line('x', 'y', source=source, line_width=2, color = '#1F78B4')
p.grid.grid_line_alpha = 0.1
p.grid[0].ticker.desired_num_ticks = 12
p.axis.major_label_text_font_size = "8pt"
p.axis.axis_label_text_font_size = "8pt"
p.axis.axis_label_text_font_style = "bold"

# CustomJS script to update plot

This following section is unique to Bokeh callbacks in the jupyter notebook. The callbak function is called every time the user adjusts the slider and will run the cb_code javascript to udpate the x and y variables. Note that a strange nuance is that as x and y variables are updated, the ColumnDataSource variable is also indirectly update and this seems to be unique to Bokeh as far as we've seen.

In [4]:
# showing controls and plot
exp_slider = Slider(start=-20, end=20, value=1, step=.1, title="Exponent parameter i ")
layout = vform(p, exp_slider)

cb_code =  """
        var data = source.get('data');
        var exp = slider_obj.get('value');
        x = data['x'];
        y = data['y'];
        for(i = 0; i < x.length ; i++)
        {
            y[i] = Math.pow((x[i]) , exp);
        }

        source.trigger('change');
    """
# string for javascript callback function, taken from bokeh documentation
#This needs to be placed after the exp_slider widget has been defined
cb1 = CustomJS(args=dict(source=source, slider_obj=exp_slider), code=cb_code)

#IMPORTANT: defining the callback for the slider needs to be done AFTER the callback
#has been defined, or else the slider won't reference the callback correctly
exp_slider.callback = cb1

show(layout)


# Another example of plotting sine wave with varying amplitude, angular frequency, and phase

Using the same principles as the plot above, we can now generate any plot with varying parameters controlled by sliders. Here, we demonstrated plotting $Asin(\omega x + t)$, where the amplitude, phase, and angular frequency are adjustable by the user.

In [5]:
# defining data source and x, y variables to plot
# initial value for x and y vectors
start = -10
end = 10
num_points = 1000
x_1 = np.linspace(start, end, num_points)
y_1 = np.sin(x_1)
# defining data source for Bokeh's ColumnDataSource data type
source_1 = ColumnDataSource(data=dict(x=x_1, y=y_1))

In [6]:
# plotting and adjusting figure properties
p_1 = figure(plot_width=500, plot_height=500, title="Plot of a*sin(wx + t)", toolbar_location="below", tools="pan,wheel_zoom,box_zoom,reset,resize")
p_1.line('x', 'y', source=source_1, line_width=2, color = '#1F78B4')
p_1.grid.grid_line_alpha = 0.1
p_1.grid[0].ticker.desired_num_ticks = 12
p_1.axis.major_label_text_font_size = "8pt"
p_1.axis.axis_label_text_font_size = "8pt"
p_1.axis.axis_label_text_font_style = "bold"
show(p_1)


In [7]:
a_slider = Slider(start=-10, end=10, value=1, step=.01, title="Amplitude a")
w_slider = Slider(start=-10, end=10, value=1, step=.01, title="Angular Frequency w") 
t_slider = Slider(start=-10, end=10, value=1, step=.01, title="Phase t")


cb_code_1 =  """
        var data = source.get('data');
        var a = a_slider.get('value');
        var w = w_slider.get('value');
        var t = t_slider.get('value');
        x_1 = data['x'];
        y_1 = data['y'];
        for(i = 0; i < x.length ; i++)
        {
            if(x_1[i] != 0)
            {
                y_1[i] = a*Math.sin(w*(x_1[i])+t)
            }
        }

        source.trigger('change');
    """

# string for javascript callback function, taken from bokeh documentation
#This needs to be placed after the exp_slider widget has been defined
cb_sinc = CustomJS(args=dict(source=source_1, a_slider=a_slider, w_slider=w_slider, t_slider=t_slider,), code=cb_code_1)

#IMPORTANT: defining the callback for the slider needs to be done AFTER the callback
#has been defined, or else the slider won't reference the callback correctly
a_slider.callback = cb_sinc
w_slider.callback = cb_sinc
t_slider.callback = cb_sinc

In [8]:
layout_1 = vform(p_1, a_slider, w_slider, t_slider)
show(layout_1)